In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

FOLDER = '10_epochs'
RESULT_PATH = os.path.join(FOLDER, 'tuning_results.csv')

In [ ]:
original_df = pd.read_csv(RESULT_PATH)
original_df.sort_values('val_acc', ascending=False).head()

In [ ]:
def create_norm_column(df, layer_column, batch_column):
    series = pd.Series(data=['' for _ in range(len(df))])
    series[df[layer_column]] = 'layer'
    series[df[batch_column]] = 'batch'
    series[series == ''] = 'none'
    return series

df = original_df.copy()
df['class_norm'] = create_norm_column(df, 'class_use_laynorm_inp', 'class_use_batchnorm_inp')
df['fc_norm'] = create_norm_column(df, 'fc_use_laynorm_inp', 'fc_use_batchnorm_inp')
df['fc_norm'] += ',' + create_norm_column(df, 'fc_use_laynorm', 'fc_use_batchnorm')
df['cnn_norm'] = create_norm_column(df, 'cnn_use_laynorm_inp', 'cnn_use_batchnorm_inp')
df['cnn_norm'] += ',' + create_norm_column(df, 'cnn_use_laynorm', 'cnn_use_batchnorm')

boolean_columns = list(filter(lambda c: '_use_' in c, df.columns))
df = df.drop(columns=boolean_columns)
df = df[[
    'uid',
    'cnn_norm', 'cnn_drop', 'cnn_act',
    'fc_norm', 'fc_lay', 'fc_drop', 'fc_act',
    'class_norm', 'lr', 'optimizer',
    'train_acc', 'val_acc'
]]
df['rel_diff'] = abs(df['train_acc'] - df['val_acc']) / df[['train_acc', 'val_acc']].max(axis=1)
df = df.sort_values('val_acc', ascending=False)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[df['cnn_norm'].map(lambda n: n.startswith('layer,'))]

In [ ]:
def draw_loss_acc_plots(uid):
    log_path = os.path.join(FOLDER, uid, 'log.csv')
    plot_df = pd.read_csv(log_path)
    fig, axs = plt.subplots(2, sharex=True)
    fig.suptitle(uid[:8])
    for value, ax in zip(['accuracy', 'loss'], axs):
        ax.plot(plot_df[value], label='train')
        ax.plot(plot_df['val_{}'.format(value)], label='val')
        ax.set(xlabel='epochs', ylabel=value)
        ax.legend()
    plt.show()


for uid in df['uid'].tail(20):
    draw_loss_acc_plots(uid)